In [2]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime

import talib
import joblib
import os

In [3]:
# 連接 MT5
if not mt5.initialize():
    print("初始化失敗")
    mt5.shutdown()

# 獲取 Market Watch 內的所有交易品種
symbols = mt5.symbols_get()
symbol_list = [symbol.name for symbol in symbols]

print("Market Watch 內的品種數量:", len(symbol_list))
print("品種列表:", symbol_list)

# 斷開連線
mt5.shutdown()

Market Watch 內的品種數量: 166
品種列表: ['XTIUSD.x', 'XAGUSD.x', 'AMD', 'AVGO', 'BA', 'EBAY', 'GE', 'GOOG', 'GOOGL', 'JNJ', 'NFLX', 'RTX', 'TSLA', 'UNH', 'DAX30', 'HSI50', 'NAS100', 'NK225', 'SPX500', 'US30', 'XAUUSD.x', 'XBRUSD.x', 'AUDCAD.x', 'AUDCHF.x', 'AUDJPY.x', 'AUDNZD.x', 'AUDUSD.x', 'CADCHF.x', 'CADJPY.x', 'CHFJPY.x', 'EURAUD.x', 'EURCAD.x', 'EURCHF.x', 'EURGBP.x', 'EURJPY.x', 'EURNZD.x', 'EURUSD.x', 'GBPAUD.x', 'GBPCAD.x', 'GBPCHF.x', 'GBPJPY.x', 'GBPNZD.x', 'GBPUSD.x', 'NZDCAD.x', 'NZDCHF.x', 'NZDJPY.x', 'NZDUSD.x', 'USDCAD.x', 'USDCHF.x', 'USDCNH.x', 'USDHKD.x', 'USDJPY.x', 'USDSEK.x', 'USDSGD.x', 'USDZAR.x', 'AIG', 'APA', 'BRKb', 'EOG', 'FLEX', 'GM', 'JBL', 'LMT', 'SLB', 'UPS', 'QQQ', 'VIXY', 'ABBV', 'ABT', 'CAT', 'CMCSA', 'COST', 'CRM', 'CVX', 'DIS', 'DOW', 'FDX', 'IBM', 'LOW', 'MA', 'MMM', 'MS', 'NEE', 'NKE', 'PYPL', 'TGT', 'V', 'WBA', 'WFC', 'ADBE', 'A', 'AMZN', 'AXP', 'AMGN', 'ADI', 'AAPL', 'AMAT', 'T', 'BAC', 'BBY', 'BIIB', 'BMY', 'BSX', 'CHKP', 'CIEN', 'CRUS', 'CSCO', 'C', 'K

True

In [4]:
# 獲取歷史數據

data = {}
timeframe = mt5.TIMEFRAME_M30
n_candles = 1000

for symbol in symbol_list:
    mt5.initialize()
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000)
    if rates is not None:
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        data[symbol] = df
        print(f"{symbol} 歷史數據獲取成功")
    else:    
        print(f"{symbol} 歷史數據獲取失敗")

    mt5.shutdown()

XTIUSD.x 歷史數據獲取成功
XAGUSD.x 歷史數據獲取成功
AMD 歷史數據獲取成功
AVGO 歷史數據獲取成功
BA 歷史數據獲取成功
EBAY 歷史數據獲取成功
GE 歷史數據獲取成功
GOOG 歷史數據獲取成功
GOOGL 歷史數據獲取成功
JNJ 歷史數據獲取成功
NFLX 歷史數據獲取成功
RTX 歷史數據獲取成功
TSLA 歷史數據獲取成功
UNH 歷史數據獲取成功
DAX30 歷史數據獲取成功
HSI50 歷史數據獲取成功
NAS100 歷史數據獲取成功
NK225 歷史數據獲取成功
SPX500 歷史數據獲取成功
US30 歷史數據獲取成功
XAUUSD.x 歷史數據獲取成功
XBRUSD.x 歷史數據獲取成功
AUDCAD.x 歷史數據獲取成功
AUDCHF.x 歷史數據獲取成功
AUDJPY.x 歷史數據獲取成功
AUDNZD.x 歷史數據獲取成功
AUDUSD.x 歷史數據獲取成功
CADCHF.x 歷史數據獲取成功
CADJPY.x 歷史數據獲取成功
CHFJPY.x 歷史數據獲取成功
EURAUD.x 歷史數據獲取成功
EURCAD.x 歷史數據獲取成功
EURCHF.x 歷史數據獲取成功
EURGBP.x 歷史數據獲取成功
EURJPY.x 歷史數據獲取成功
EURNZD.x 歷史數據獲取成功
EURUSD.x 歷史數據獲取成功
GBPAUD.x 歷史數據獲取成功
GBPCAD.x 歷史數據獲取成功
GBPCHF.x 歷史數據獲取成功
GBPJPY.x 歷史數據獲取成功
GBPNZD.x 歷史數據獲取成功
GBPUSD.x 歷史數據獲取成功
NZDCAD.x 歷史數據獲取成功
NZDCHF.x 歷史數據獲取成功
NZDJPY.x 歷史數據獲取成功
NZDUSD.x 歷史數據獲取成功
USDCAD.x 歷史數據獲取成功
USDCHF.x 歷史數據獲取成功
USDCNH.x 歷史數據獲取成功
USDHKD.x 歷史數據獲取成功
USDJPY.x 歷史數據獲取成功
USDSEK.x 歷史數據獲取成功
USDSGD.x 歷史數據獲取成功
USDZAR.x 歷史數據獲取成功
AIG 歷史數據獲取成功
APA 歷史數據獲取成功
BRKb 歷史數據獲取成功
EOG 歷史數據獲取成功
FLEX 歷史數據獲取成功
GM 歷史數據獲取成功
JBL

In [60]:
data['XOM']

,time,open,high,low,close,tick_volume,spread,real_volume
0,2024-10-29 22:30:00,118.911,119.121,118.551,118.561,1179,78,0
1,2024-10-29 23:00:00,118.561,118.661,117.801,117.821,1257,78,0
2,2024-10-29 23:30:00,117.811,118.091,117.651,118.031,1119,78,0
3,2024-10-30 00:00:00,118.041,118.151,117.881,117.911,1038,78,0
4,2024-10-30 00:30:00,117.901,117.901,117.351,117.601,946,78,0
...,...,...,...,...,...,...,...,...
995,2025-02-22 02:30:00,110.861,111.011,110.611,110.721,520,58,0
996,2025-02-22 03:00:00,110.711,111.071,110.521,111.001,532,58,0
997,2025-02-22 03:30:00,111.001,111.041,110.741,110.911,506,58,0
998,2025-02-22 04:00:00,110.891,110.931,110.671,110.721,538,58,0


In [5]:
# 技術指標
def add_indicator(df):
    df['SMA_10'] = talib.SMA(df['close'], timeperiod=10)
    df['SMA_50'] = talib.SMA(df['close'], timeperiod=50)
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    return df


In [6]:
# 為所有貨幣對添加技術指標
for symbol in data:
    data[symbol] = add_indicator(data[symbol])

# 檢視 EURUSD 的數據
data["XTIUSD.x"].tail()

,time,open,high,low,close,tick_volume,spread,real_volume,SMA_10,SMA_50,RSI,ATR
995,2025-02-24 20:30:00,70.42,70.48,70.23,70.27,235,4,0,70.351,70.5880,44.851682,0.184327
996,2025-02-24 21:00:00,70.27,70.35,70.14,70.22,297,4,0,70.351,70.5570,42.985424,0.186161
997,2025-02-24 21:30:00,70.22,70.37,70.15,70.18,351,4,0,70.340,70.5266,41.497801,0.188578
998,2025-02-24 22:00:00,70.18,70.53,70.15,70.46,672,4,0,70.359,70.5004,53.602385,0.202251
999,2025-02-24 22:30:00,70.47,70.51,70.44,70.49,90,4,0,70.387,70.4736,54.684255,0.192804


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

model_path = "../models"

for symbol in symbol_list:
    if symbol not in [file_name.replace(".pkl", "") for file_name in os.listdir(model_path)]:
        # 設定訓練數據
        X = data[symbol][['SMA_10', 'SMA_50', 'RSI', 'ATR', 'open', 'high', 'low', 'close', 'tick_volume']].dropna()
        # 預測下一根 K 線的收盤價
        y = data[symbol]['close'].shift(-1).dropna()


        # 對齊 X 和 y
        # 最後一天不會有 y 值，所以要刪除
        X = X.iloc[:-1]
        # 前 48 個數值因為計算SMA50，所以沒有X值要刪除
        y = y.iloc[48:-1]

        # 拆分訓練和測試數據
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 訓練隨機森林模型
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # 預測
        y_pred = model.predict(X_test)

        # 模型評估
        from sklearn.metrics import mean_absolute_error
        mae = mean_absolute_error(y_test, y_pred)
        print(f"{symbol} MAE: {mae}")

        # 檢視預測結果
        y_test = y_test.reset_index(drop=True)
        y_pred = pd.Series(y_pred)

        result = pd.concat([y_test, y_pred], axis=1)
        result.columns = ['actual', 'predicted']
        display(result.head())

        # 保存模型
        joblib.dump(model, f"{model_path}/{symbol}_model.pkl")

XTIUSD.x MAE: 0.00463421052631488


,actual,predicted
0,73.05,73.0392
1,70.13,70.1184
2,70.44,70.4466
3,72.23,72.2331
4,72.63,72.6250


XAGUSD.x MAE: 0.004342421052629562


,actual,predicted
0,31.400,31.40481
1,32.671,32.67076
2,32.118,32.11950
3,32.837,32.83341
4,32.931,32.93003


AMD MAE: 0.07249478947367728


...

,actual,predicted
0,109.471,109.46931
1,107.840,107.83381
2,107.190,107.18740
3,108.230,108.22994
4,107.530,107.52780


EWJ MAE: 0.007973684210534097


,actual,predicted
0,68.297,68.29878
1,69.640,69.63712
2,68.540,68.54089
3,68.930,68.92190
4,69.040,69.04220


IYR MAE: 0.017876368421047655


,actual,predicted
0,101.539,101.50644
1,96.430,96.42700
2,95.230,95.21420
3,95.790,95.78890
4,95.850,95.84830


In [42]:
print(len(X))
print(len(y))

951
951
